In [76]:
using Distributions
using KernelDensity
using Plots
plotly()

Plots.PlotlyBackend()

In [25]:
mu_0 = 5.0 # avg income from fishing
mu_1 = 6.0 # avg income from hunting
mu_Y = [mu_1, mu_0] # mean vector for Y
mu_U = [0.0, 0.0]
sigma = [1.0 -0.5; -0.5 2.0] # covariance matrix for Y and U

Y = MvNormal(mu_Y, sigma)
U = MvNormal(mu_U, sigma)

FullNormal(
dim: 2
μ: [0.0,0.0]
Σ: 2x2 Array{Float64,2}:
  1.0  -0.5
 -0.5   2.0
)


In [190]:
srand(123) # set seed
u = rand(U, 100)

# plot the distribution of the talents
scatter(u[1,:], u[2,:], color= :blue, label="", title="Distribution of U (talents)", xlabel="Fishing (U0)", ylabel="Hunting (U1)")
hline!([0], label="", color=:black)
vline!([0], label="", color=:black)
xlims!(-5,5)
ylims!(-5,5)

In [193]:
# Marginal distributions of f(Y|D=1) and f(Y|D=0)
Plots.plot([x -> pdf(Normal(5,2), x),
    x -> pdf(Normal(6,1), x)]
, -3, 13, label=["f(Y|D=0) income fishing" "f(Y|D=1) income hunting"], title="Marginal distributions of Y")

People select the treatment ($D=1$) if $Y_1 > Y_0$ where $$Y_1 = \mu_1 + U_1 \\ Y_0 = \mu_0 + U_0 $$

We can therefore specify for everyone in the population, if they should choose the treatment by solving

$$
\begin{align}
Y_1 - Y_0 &> 0 \\
(\mu_1 + U_1) - (\mu_0 + U_0) &> 0 \\
(\mu_1 + U_1) - (\mu_0 + U_0) &= 0 \\
U_0 &= \mu_1 - \mu_0 + U_1
\end{align}
$$

In [202]:
scatter(u[1,:], u[2,:], color= :blue, label="", title="Distribution of U (talents)", xlabel="Fishing (U0)", ylabel="Hunting (U1)")
hline!([0], label="", color=:black)
vline!([0], label="", color=:black)
xlims!(-5,5)
ylims!(-5,5)
plot!(x -> x+1, -5, 5, label="threshold for choosing D=1 (below line)")

In [223]:
stdU = sqrt(sigma[1,1] + sigma[2,2] - 2*sigma[1,2]) # this is the standard error of U
Z    = (mu_1 - mu_0) / stdU # this is the Z value
eps  = (u[2,:] - u[1,:]) / stdU # these are the epsilon values, specific for each person

# people choose D=1 if eps < Z
idx = eps .< Z
uD1 = u[:,idx];

In [224]:
scatter(u[1,:], u[2,:], color= :blue, label="", title="Distribution of U (talents)", xlabel="Fishing (U0)", ylabel="Hunting (U1)")
scatter!(uD1[1,:], uD1[2,:], color= :red, label="", title="Distribution of U (talents)", xlabel="Fishing (U0)", ylabel="Hunting (U1)")
hline!([0], label="", color=:black)
vline!([0], label="", color=:black)
xlims!(-5,5)
ylims!(-5,5)
plot!(x -> x+1, -5, 5, label="threshold for choosing D=1 (below line)")

We can summarize this result with $$ P(D=1) = P(\epsilon < Z) = \Phi(Z) $$

Similarly, we can ask ourselves: Who takes the treatment (selects hunting)? this can be visualized by the marginal distribution of hunting. Given the threshold of selecting hunting (=.5), this becomes a truncated normal distribution!

In [240]:
# Marginal distributions of f(Y|D=1) and f(Y|D=0)
Plots.plot([x -> pdf(Normal(0,1), x)]
, -4, 4, label=["f(Y|D=1) income hunting"], title="Marginal distributions of Y|D=1")
vline!([0.5], label="Z")

We can now evaluate all terms needed for ATE and ATOT:
$$
\begin{align}
E[Y_1|D=1] &= E[\mu_1 + U_1 | D = 1] \\
&= \mu_1 + E[U_1|D=1] \\
&= \mu_1 + E[U_1|\epsilon < Z] \\
&= \mu_1 - \sigma_{1\epsilon}\frac{\phi(Z)}{\Phi(Z)}
\end{align}
$$

In [262]:
sig0e = cov(u[2,:]', eps')
sig1e = cov(u[1,:]', eps')
N = Normal(0, 1);

In [266]:
EY1D1 = mu_1 - sig1e* (pdf(N, Z) / cdf(N, Z))
EY0D1 = mu_0 - sig0e* (pdf(N, Z) / cdf(N, Z))

ATOT = EY1D1 - EY0D1

1x1 Array{Float64,2}:
 2.00317

In [267]:
EY0D0 = mu_0 - sig0e* (pdf(N, Z) / (1 - cdf(N, Z)))
SB = EY0D1 - EY0D0 # selection bias

1x1 Array{Float64,2}:
 0.763565

In [269]:
NAIVE = EY1D1 - EY0D0

1x1 Array{Float64,2}:
 2.76674